In [1]:
!pip install keras-tuner

   ---------------------------------------- 0.0/129.1 kB ? eta -:--:--
   ---------------------------------------- 0.0/129.1 kB ? eta -:--:--
   --- ------------------------------------ 10.2/129.1 kB ? eta -:--:--
   ------------ -------------------------- 41.0/129.1 kB 487.6 kB/s eta 0:00:01
   ------------ -------------------------- 41.0/129.1 kB 487.6 kB/s eta 0:00:01
   ------------------------------------ - 122.9/129.1 kB 717.5 kB/s eta 0:00:01
   -------------------------------------- 129.1/129.1 kB 691.3 kB/s eta 0:00:00


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import requests

In [3]:
fashion_mnist=keras.datasets.fashion_mnist

In [4]:
(train_images , train_labels), (test_images , test_labels)=fashion_mnist.load_data()

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 2us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 7s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [5]:
train_images=train_images/255.0
test_images=test_images/255.0

In [6]:
train_images[0].shape

(28, 28)

In [7]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [8]:
def build_model(hp):
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter' , min_value=32, max_value=128 , step=16),
        kernel_size=hp.Choice('conv_1_kernel',values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter' , min_value=32, max_value=64 , step=16),
        kernel_size=hp.Choice('conv_2_kernel',values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)    
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128 , step=16),
         activation='relu',
    ),          
    keras.layers.Dense(10,activation='softmax')#output layer                            
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
       loss='sparse_categorical_crossentropy',
       metrics=['accuracy'])
  
  return model            


In [9]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

C:\Users\Admin\AppData\Local\Temp\ipykernel_9448\556418634.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import RandomSearch


In [10]:
tuner_search = RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
64                |64                |conv_1_filter
5                 |5                 |conv_1_kernel
32                |32                |conv_2_filter
5                 |5                 |conv_2_kernel
64                |64                |dense_1_units
0.001             |0.001             |learning_rate


Epoch 1/3
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 134s 78ms/step - accuracy: 0.7915 - loss: 0.5897 - val_accuracy: 0.8825 - val_loss: 0.3203
Epoch 2/3
 404/1688 ━━━━━━━━━━━━━━━━━━━━ 1:56 91ms/step - accuracy: 0.8936 - loss: 0.2939

In [ ]:
model=tuner_search.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 48)        480       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        13856     
_________________________________________________________________
flatten (Flatten)            (None, 18432)             0         
_________________________________________________________________
dense (Dense)                (None, 112)               2064496   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1130      
Total params: 2,079,962
Trainable params: 2,079,962
Non-trainable params: 0
_________________________________________________________________
